In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [0]:
import pandas as pd
link_hilstrom = 'https://drive.google.com/open?id=15osyN4c5z1pSo1JkxwL_N8bZTksRvQuU'
fluff, id = link_hilstrom.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Hillstrom.csv')
hillstrom_df = pd.read_csv('Hillstrom.csv')

In [0]:
import pandas as pd
link_ = 'https://drive.google.com/open?id=1b8N7WtwIe2WmQJD1KL5UAy70K13MxwKj'
fluff, id = link_.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Lalonde.csv')
lalonde_df = pd.read_csv('Lalonde.csv')

In [0]:
import csv
import json
import os
from os.path import isfile, join
from sklearn.model_selection import KFold, StratifiedKFold


def preprocess_data(df, dataset='hillstrom', verbose=True):
    """
    Preprocessing the dataset
     - Use one-hot encoding for categorical features
     - Check the name of the target variable and treatment variable
     - Drop the unused columns
     - Delete the unused data
    
    Args:
        df: A pandas.DataFrame which have all data of the dataset
        dataset: the name of the dataset
    Return:
        # I recommend to split into the dataframes of predictor variables, the 
        # target variable, and the treatment varaible
        # df_x: the dataframes of predictor variables
        # s_y: target variables
        # s_t: treatment variables
    """
    if dataset in ['hillstrom', 'email']:
        # For Hillstrom dataset, the ‘‘visit’’ target variable was selected
        #   as the target variable of interest and the selected treatment is 
        #   the e-mail campaign for women’s merchandise [1]
        # [1] Kane K, Lo VSY, Zheng J. True-lift modeling: Comparison of methods. 
        #    J Market Anal. 2014;2:218–238
    
        # Delete unused data: men's email cases should be removed
        df_x = df[df.segment != 'Mens E-Mail'].reset_index()
        # Assign Y for target (visit: 0, 1)
        s_y = df_x['visit']
        # Assign T for treatment (segment: Womens E-Mail, Mens E-Mail (not used), No E-Mail)
        s_t = (df_x['segment'] == 'Womens E-Mail').astype('int64')
        # Drop unused columns from X
        df_x = df_x.drop(columns=['conversion', 'spend', 'visit', 'segment', 'index'])
        # One-hot encoding for categorical features
        df_x = pd.get_dummies(df_x)

    elif dataset in ['criteo', 'ad']:
        raise NotImplementedError

    elif dataset in ['lalonde', 'job']:
        # Delete unused data: None
        df_x = df.reset_index()
        # Target variables (RE78: earnings in 1978)
        s_y = df_x['RE78']
        # Treatment variables (treatment: 0, 1)
        s_t = df_x['treatment']
        # Drop unused columns
        df_x = df_x.drop(columns=['treatment', 'RE78', 'index'])
        # One-hot encoding for categorical features
        df_x = pd.get_dummies(df_x)

    else:
        raise NotImplementedError

    return df_x, s_y, s_t

In [7]:
def print_preproccessing_data(df_x, s_y, s_t):
    print(df_x[:5])
    print(df_x.columns.values)
    print(df_x.shape)
    print(s_y[:5])
    print(s_t[:5])


hillstrom_df_x, hillstrom_s_y, hillstrom_s_t = preprocess_data(hillstrom_df, 'hillstrom')
print_preproccessing_data(hillstrom_df_x, hillstrom_s_y, hillstrom_s_t)

lalonde_df_x, lalonde_s_y, lalonde_s_t = preprocess_data(lalonde_df, 'lalonde')
print_preproccessing_data(lalonde_df_x, lalonde_s_y, lalonde_s_t)

"""
def print_preproccessing_data(df):
    print(df[:5])
    print(df.columns.values)
    print(df.shape)
    print(df['Y'][:5])
    print(df['T'][:5])


hillstrom_df_pre = preprocess_data(hillstrom_df, 'hillstrom')
print_preproccessing_data(hillstrom_df_pre)

lalonde_df_pre = preprocess_data(lalonde_df, 'lalonde')
print_preproccessing_data(lalonde_df_pre)
"""

   recency  history  mens  ...  channel_Multichannel  channel_Phone  channel_Web
0       10   142.44     1  ...                     0              1            0
1        6   329.08     1  ...                     0              0            1
2        7   180.65     0  ...                     0              0            1
3        2    45.34     1  ...                     0              0            1
4        6   134.83     0  ...                     0              1            0

[5 rows x 18 columns]
['recency' 'history' 'mens' 'womens' 'newbie'
 'history_segment_1) $0 - $100' 'history_segment_2) $100 - $200'
 'history_segment_3) $200 - $350' 'history_segment_4) $350 - $500'
 'history_segment_5) $500 - $750' 'history_segment_6) $750 - $1,000'
 'history_segment_7) $1,000 +' 'zip_code_Rural' 'zip_code_Surburban'
 'zip_code_Urban' 'channel_Multichannel' 'channel_Phone' 'channel_Web']
(42693, 18)
0    0
1    0
2    0
3    0
4    1
Name: visit, dtype: int64
0    1
1    0
2    1
3    1
4 

"\ndef print_preproccessing_data(df):\n    print(df[:5])\n    print(df.columns.values)\n    print(df.shape)\n    print(df['Y'][:5])\n    print(df['T'][:5])\n\n\nhillstrom_df_pre = preprocess_data(hillstrom_df, 'hillstrom')\nprint_preproccessing_data(hillstrom_df_pre)\n\nlalonde_df_pre = preprocess_data(lalonde_df, 'lalonde')\nprint_preproccessing_data(lalonde_df_pre)\n"

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

def performance(pr_y1_ct1, pr_y1_ct0, y, ct, groups=10):
    """
    1. Split the total customers into the given number of groups
    2. Calculate the statistics of each segment
    
    Args:
        pr_y1_ct1: the series (list) of the customer's expected return
        pr_y1_ct0: the expected return when a customer is not treated
        y: the observed return of customers
        ct: whether each customer is treated or not
        groups: the number of groups (segments). Should be 5, 10, or 20
    Return:
        DataFrame:
            columns:
                'n_y1_ct1': the number of treated responders
                'n_y1_ct0': the number of not treated responders
                'r_y1_ct1': the average return of treated customers
                'r_y1_ct0': the average return of not treated customers
                'n_ct1': the number of treated customers
                'n_ct0': the number of not treated customers
                'uplift': the average uplift (the average treatment effect)
            rows: the index of groups
    """

    ### check valid arguments
    if groups not in [5, 10, 20]:
        raise Exception("uplift: groups must be either 5, 10 or 20")

    ### check for NAs.
    if pr_y1_ct1.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_ct1")
    if pr_y1_ct0.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_ct0")
    if y.isnull().values.any():
        raise Exception("uplift: NA not permitted in y")
    if ct.isnull().values.any():
        raise Exception("uplift: NA not permitted in ct")

    ### check valid values for ct
    if set(ct) != {0, 1}:
        raise Exception("uplift: ct must be either 0 or 1")

    ### check length of arguments
    if not (len(pr_y1_ct1) == len(pr_y1_ct0) == len(y) == len(ct)):
        raise Exception("uplift: arguments pr_y1_ct1, pr_y1_ct0, y and ct must all have the same length")

    ###############################
    ###     Do it yourself!     ###
    ###############################
    # tr: treated responder, cr: controlled responder
    # Customers ordered by predicted uplift values in descending order are segmented.
    df = pd.DataFrame(data={'ct': ct, 'y': y,
                            'uplift_rank': (pr_y1_ct1 - pr_y1_ct0).rank(ascending=False, method='first')})
    
    df_group = df.groupby(pd.qcut(df['uplift_rank'], groups, labels=range(1, groups + 1)))
    
    # Get group data
    n_ct1 = df_group['ct'].sum()
    n_ct0 = df_group.size() - n_ct1
    n_y1_ct1 = df_group.apply(lambda r: r[r['ct'] == 1]['y'].sum())
    n_y1_ct0 = df_group.apply(lambda r: r[r['ct'] == 0]['y'].sum())
    r_y1_ct1 = n_y1_ct1 / n_ct1
    r_y1_ct0 = n_y1_ct0 / n_ct0
    uplift = r_y1_ct1 - r_y1_ct0
    
    return pd.DataFrame(data={'n_y1_ct1': n_y1_ct1, 'n_y1_ct0': n_y1_ct0,
                              'r_y1_ct1': r_y1_ct1, 'r_y1_ct0': r_y1_ct0,
                              'n_ct1': n_ct1, 'n_ct0': n_ct0, 'uplift': uplift})


def qini(perf, plotit=True):
    nrow = len(perf)

    # Calculating the incremental gains. 
    # - First, the cumulitative sum of the treated and the control groups are
    #  calculated with respect to the total population in each group at the
    #  specified decile
    # - Afterwards we calculate the percentage of the total amount of people
    #  (both treatment and control) are present in each decile
    cumul_y1_t1 = (perf['n_y1_t1'].cumsum() / perf['n_t1'].cumsum()).fillna(0)
    cumul_y1_t0 = (perf['n_y1_t0'].cumsum() / perf['n_t0'].cumsum()).fillna(0)
    deciles = [i/nrow for i in range(1, nrow+1)]

    ### Model Incremental gains
    inc_gains = (cumul_y1_t1 - cumul_y1_t0) * deciles
    inc_gains = [0.0] + list(inc_gains)

    ### Overall incremental gains
    overall_inc_gain = sum(perf['n_y1_t1']) / sum(perf['n_t1']) \
            - sum(perf['n_y1_t0']) / sum(perf['n_t0'])

    ### Random incremental gains
    random_inc_gains = [i*overall_inc_gain / nrow for i in range(nrow+1)]

    ### Compute area under the model incremental gains (uplift) curve
    x = [0] + deciles
    y = list(inc_gains)
    auuc = 0
    auuc_rand = 0

    auuc_list = [auuc]
    for i in range(1, len(x)):
        auuc += 0.5 * (x[i] - x[i-1]) * (y[i] + y[i-1])
        auuc_list.append(auuc)

    ### Compute area under the random incremental gains curve
    y_rand = random_inc_gains

    auuc_rand_list = [auuc_rand]
    for i in range(1, len(x)):
        auuc_rand += 0.5 * (x[i] - x[i-1]) * (y_rand[i] + y_rand[i-1])
        auuc_rand_list.append(auuc_rand)

    ### Compute the difference between the areas (Qini coefficient)
    Qini = auuc - auuc_rand

    ### Plot incremental gains curve
    if plotit:
        x_axis = x
        plt.plot(x_axis, inc_gains)
        plt.plot(x_axis, random_inc_gains)
        plt.show()
    
    ### Qini 30%, Qini 10%
    n_30p = int(nrow*3/10)
    n_10p = int(nrow/10)
    qini_30p = auuc_list[n_30p] - auuc_rand_list[n_30p]
    qini_10p = auuc_list[n_10p] - auuc_rand_list[n_10p]

    res = {
        'qini': Qini,
        'inc_gains': inc_gains,
        'random_inc_gains': random_inc_gains,
        'auuc_list': auuc_list,
        'auuc_rand_list': auuc_rand_list,
        'qini_30p': qini_30p,
        'qini_10p': qini_10p,
    }    

    return res


def qini2(perf, plotit=True):
    """
    Calculating the incremental gains (y-axis of Qini curve)
     - First, the cumulitative sum of the treated and the control groups are
      calculated with respect to the total population in each group at the
      specified decile
     - Afterwards we calculate the percentage of the total amount of people
      (both treatment and control) are present in each decile
    Args:
        perf: A return of the performance function (above)
        plotit: whether draw a plot or not
    Return:
        1. Qini value
        2. return or save the plot if plotit is True
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    # Get qini value
    s_rt = perf['r_y1_ct1'].cumsum() / perf['n_y1_ct1'].cumsum()
    s_rc = perf['r_y1_ct0'].cumsum() / perf['n_y1_ct0'].cumsum()
    k = len(s_rt)
    qini_value = s_rt * s_rt.index / k

    # Draw qini curve
    plt.rcParams['figure.figsize'] = (5.0, 4.0)
    plt.plot(qini_value, label='qini')
    plt.legend(['Bad model'])
    plt.show()
    
    return qini_value


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def performance(pr_y1_t1, pr_y1_t0, y, t, groups=10):
    """
    1. Split the total customers into the given number of groups
    2. Calculate the statistics of each segment
    
    Args:
        pr_y1_t1: the series (list) of the customer's expected return
        pr_y1_t0: the expected return when a customer is not treated
        y: the observed return of customers
        t: whther each customer is treated or not
        groups: the number of groups (segments). Should be 5, 10, or 20
    Return:
        DataFrame:
            columns:
                'n_y1_t1': the number of treated responders
                'n_y1_t0': the number of not treated responders
                'r_y1_t1': the average return of treated customers
                'r_y1_t0': the average return of not treated customers
                'n_t1': the number of treated customers
                'n_t0': the number of not treated customers
                'uplift': the average uplift (the average treatment effect)
            rows: the index of groups
    """
  
    ### check valid arguments
    if groups not in [5, 10, 20]:
        raise Exception("uplift: groups must be either 5, 10 or 20")
  
    ### check for NAs.
    if pr_y1_t1.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_t1")
    if pr_y1_t0.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_t0")
    if y.isnull().values.any():
        raise Exception("uplift: NA not permitted in y")
    if t.isnull().values.any():
        raise Exception("uplift: NA not permitted in t")
   
    ### check valid values for y and t
    # if set(y) != {0, 1}:
    #     raise Exception("uplift: y must be either 0 or 1")
    if set(t) != {0, 1}:
        raise Exception("uplift: t must be either 0 or 1")

    ### check length of arguments
    if not (len(pr_y1_t1) == len(pr_y1_t0) == len(y) == len(t)):
        raise Exception("uplift: arguments pr_y1_t1, pr_y1_t0, y and t must all have the same length")

    ### define dif_pred
    dif_pred = pr_y1_t1 - pr_y1_t0
  
    ### Make index same
    y.index = dif_pred.index
    t.index = dif_pred.index
    
    mm = pd.DataFrame({
        'dif_pred': dif_pred,
        'y': y,
        't': t,
        'dif_pred_r': dif_pred.rank(ascending=False, method='first')
    })

    mm_groupby = mm.groupby(pd.qcut(mm['dif_pred_r'], groups, labels=range(1, groups+1), duplicates='drop'))
  
    n_y1_t1 = mm_groupby.apply(lambda r: r[r['t'] == 1]['y'].sum())
    n_y1_t0 = mm_groupby.apply(lambda r: r[r['t'] == 0]['y'].sum())
    n_t1 = mm_groupby['t'].sum()
    n_t0 = mm_groupby['t'].count() - n_t1
  
    df = pd.DataFrame({
        'n_t1': n_t1,
        'n_t0': n_t0,
        'n_y1_t1': n_y1_t1,
        'n_y1_t0': n_y1_t0,
        'r_y1_t1': n_y1_t1 / n_t1,
        'r_y1_t0': n_y1_t0 / n_t0,
    })
    fillna_columns = ['n_y1_t1', 'n_y1_t0', 'n_t1', 'n_t0']
    df[fillna_columns] = df[fillna_columns].fillna(0)
    df.index.name = 'groups'

    df['uplift'] = df['r_y1_t1'] - df['r_y1_t0']
    df['uplift'] = round(df['uplift'], 6)

    return df


def qini(perf, plotit=True):
    nrow = len(perf)

    # Calculating the incremental gains. 
    # - First, the cumulitative sum of the treated and the control groups are
    #  calculated with respect to the total population in each group at the
    #  specified decile
    # - Afterwards we calculate the percentage of the total amount of people
    #  (both treatment and control) are present in each decile
    cumul_y1_t1 = (perf['n_y1_t1'].cumsum() / perf['n_t1'].cumsum()).fillna(0)
    cumul_y1_t0 = (perf['n_y1_t0'].cumsum() / perf['n_t0'].cumsum()).fillna(0)
    deciles = [i/nrow for i in range(1, nrow+1)]

    ### Model Incremental gains
    inc_gains = (cumul_y1_t1 - cumul_y1_t0) * deciles
    inc_gains = [0.0] + list(inc_gains)

    ### Overall incremental gains
    overall_inc_gain = sum(perf['n_y1_t1']) / sum(perf['n_t1']) \
            - sum(perf['n_y1_t0']) / sum(perf['n_t0'])

    ### Random incremental gains
    random_inc_gains = [i*overall_inc_gain / nrow for i in range(nrow+1)]

    ### Compute area under the model incremental gains (uplift) curve
    x = [0] + deciles
    y = list(inc_gains)
    auuc = 0
    auuc_rand = 0

    auuc_list = [auuc]
    for i in range(1, len(x)):
        auuc += 0.5 * (x[i] - x[i-1]) * (y[i] + y[i-1])
        auuc_list.append(auuc)

    ### Compute area under the random incremental gains curve
    y_rand = random_inc_gains

    auuc_rand_list = [auuc_rand]
    for i in range(1, len(x)):
        auuc_rand += 0.5 * (x[i] - x[i-1]) * (y_rand[i] + y_rand[i-1])
        auuc_rand_list.append(auuc_rand)

    ### Compute the difference between the areas (Qini coefficient)
    Qini = auuc - auuc_rand

    ### Plot incremental gains curve
    if plotit:
        x_axis = x
        plt.plot(x_axis, inc_gains)
        plt.plot(x_axis, random_inc_gains)
        plt.show()
    
    ### Qini 30%, Qini 10%
    n_30p = int(nrow*3/10)
    n_10p = int(nrow/10)
    qini_30p = auuc_list[n_30p] - auuc_rand_list[n_30p]
    qini_10p = auuc_list[n_10p] - auuc_rand_list[n_10p]

    res = {
        'qini': Qini,
        'inc_gains': inc_gains,
        'random_inc_gains': random_inc_gains,
        'auuc_list': auuc_list,
        'auuc_rand_list': auuc_rand_list,
        'qini_30p': qini_30p,
        'qini_10p': qini_10p,
    }    

    return res

In [10]:
import itertools
import numpy as np

def parameter_tuning(fit_mdl, pred_mdl, data, search_space):
    """
    Given a model, search all combination of parameter sets and find
    the best parameter set
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "ct_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "ct_test": treatment variables of test (usually, validation) dataset,
            }
        search_space:
            {
                parameter_name: [search values]
            }
    Return:
        The best parameter set
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##

  
def wrapper(fit_mdl, pred_mdl, data)
    """
    General wrapper approach
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "ct_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "ct_test": treatment variables of test (usually, validation) dataset,
            }
    Return:
        (A list of best models, The list of dropped variables)
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##


SyntaxError: ignored

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


def tma(x, y, ct, method=LogisticRegression, **kwargs):
    """Training a model according to the "Two Model Approach" 
    (a.k.a. "Separate Model Approach")
    The default model is General Linear Model (GLM)
    
    Source: "Incremental Value Modeling" (Hansotia, 2002)

    Args:
        x: A data frame of predictors.
        y: A binary response (numeric) vector.
        ct: A binary response (numeric) representing the treatment assignment
            (coded as 0/1).
        method: A sklearn model specifying which classification or regression
            model to use. This should be a method that can handle a 
            multinominal class variable.

    Return:
        Dictionary: A dictionary of two models. One for the treatment group, 
            one for the control group.

            {
                'model_treat': a model for the treatment group,
                'model_control': a model for the control group
            }

    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return {
        'model_treat': method(random_state=1234, solver='newton-cg', C=10e8, **kwargs).fit(x[ct == 1], y[ct == 1]),
        'model_control': method(random_state=1234, solver='newton-cg', C=10e8, **kwargs).fit(x[ct == 0], y[ct == 0])
    }


def predict_tma(obj, newdata, **kwargs):
    """Predictions according to the "Two Model Approach" 
    (a.k.a. "Separate Model Approach")
    
    For each instance in newdata two predictions are made:
    1) What is the probability of a person responding when treated?
    2) What is the probability of a person responding when not treated
      (i.e. part of control group)?

    Source: "Incremental Value Modeling" (Hansotia, 2002)

    Args:
        obj: A dictionary of two models. 
            One for the treatment group, one for the control group.
        newdata: A data frame containing the values at which predictions
            are required.
    
    Return:
        DataFrame: A dataframe with predicted returns for when the customers
            are treated and for when they are not treated.
            'pr_y1_ct1': when treated, 'pr_y1_ct0': when not treated
    """

    ###############################
    ###     Do it yourself!     ###
    ###############################
    # LogisticRegression: use predict_proba and return result[:, 1] for True class
    # LinearRegression: use predict
    if isinstance(obj['model_treat'], LogisticRegression):
        return pd.DataFrame(data={
            'pr_y1_ct1': obj['model_treat'].predict_proba(newdata)[:, 1],
            'pr_y1_ct0': obj['model_control'].predict_proba(newdata)[: , 1]
        })
    else:
        return pd.DataFrame(data={
            'pr_y1_ct1': obj['model_treat'].predict(newdata),
            'pr_y1_ct0': obj['model_control'].predict(newdata)
        })

In [154]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import StratifiedKFold, KFold

def main():
    ### Hyper parameters ###
    dataset = 'hillstrom'
    seed = 1234
    n_fold = 5
    models = {
        'tma': {'model': tma, 'predict': predict_tma}
    }

    ### Load data & K fold validation ###
    if dataset == 'hillstrom':
        df_x, s_y, s_t = preprocess_data(hillstrom_df, dataset)
        fold_split = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed).split(df_x, s_y)
    elif dataset == 'lalonde':
        df_x, s_y, s_t = preprocess_data(lalonde_df, dataset)
        fold_split = KFold(n_splits=n_fold, shuffle=True, random_state=seed).split(df_x)
    else:
        assert()

    for model_name in models:
        for train_index, test_index in fold_split:
            train_x = df_x.reindex(train_index)
            train_y = s_y.reindex(train_index)
            train_t = s_t.reindex(train_index)
            test_x = df_x.reindex(test_index).reset_index(drop=True)
            test_t = s_t.reindex(test_index).reset_index(drop=True)
            test_y = s_y.reindex(test_index).reset_index(drop=True)

            model = models[model_name]['model'](train_x, train_y, train_t)
            pred = models[model_name]['predict'](model, test_x)
            print(pred[: 5])
            
            perf = performance(pred['pr_y1_ct1'], pred['pr_y1_ct0'], test_y, test_t)
            print(perf[:10])
            qini_value = qini(perf)
            print(qini_value)
            assert()
            
        ### Variable selection (General wrapper approach) ###

        ### Parameter tuning ###
        # print("Model: {}\n".format(model))
        # print("Tuning space: \n")
        # for key, val in search_space.items():
        #    print("    '{}': {}\n".format(key, val))
        # print("Seed: {}\n".format(seed))
        # print("Qini value: mean = {}, std = {}\n\n".format(mean_qini, std_qini))
        
main()


   pr_y1_ct1  pr_y1_ct0
0   0.062463   0.045249
1   0.172132   0.076263
2   0.160091   0.099979
3   0.142815   0.072647
4   0.071596   0.055533
             n_y1_ct1  n_y1_ct0  r_y1_ct1  r_y1_ct0  n_ct1  n_ct0    uplift
uplift_rank                                                                
1                  92        54  0.209567  0.130120    439    415  0.079447
2                  85        40  0.205314  0.090909    414    440  0.114405
3                  92        40  0.216981  0.093023    424    430  0.123958
4                  83        41  0.178112  0.105670    466    388  0.072441
5                  60        31  0.142518  0.071594    421    433  0.070924
6                  58        40  0.141463  0.090293    410    443  0.051170
7                  38        23  0.090692  0.052874    419    435  0.037819
8                  39        34  0.100515  0.072961    388    466  0.027554
9                  48        53  0.105727  0.132500    454    400 -0.026773
10                 7

KeyError: ignored